In this notebook, we'll be using a GRU model for a time series prediction task and we will compare the performance of the GRU model against an LSTM model as well. The dataset that we will be using is the Hourly Energy Consumption dataset which can be found on [Kaggle](https://www.kaggle.com/robikscube/hourly-energy-consumption). The dataset contains power consumption data across different regions around the United States recorded on an hourly basis.

You can run the code implementation in this article on FloydHub using their GPUs on the cloud by clicking the following link and using the main.ipynb notebook.

[![Run on FloydHub](https://static.floydhub.com/button/button-small.svg)](https://floydhub.com/run?template=https://github.com/gabrielloye/https://github.com/gabrielloye/GRU_Prediction)

This will speed up the training process significantly. Alternatively, the link to the GitHub repository can be found [here]().

The goal of this implementation is to create a model that can accurately predict the energy usage in the next hour given historical usage data. We will be using both the GRU and LSTM model to train on a set of historical data and evaluate both models on an unseen test set. To do so, we’ll start with feature selection, data-preprocessing, followed by defining, training and eventually evaluating the models.

We will be using the PyTorch library to implement both types of models along with other common Python libraries used in data analytics.

In [13]:
import os
import time
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler

# Define data root directory
#data_dir = "./data/"
#print(os.listdir(data_dir))

In [105]:
normalizeddataframe=pd.read_csv('forcetorquebuttonresults_09_21_2021.csv')#.head()
originaldataframe=normalizeddataframe

In [79]:

#normalizeddataframe


We have a total of **12** *.csv* files containing hourly energy trend data (*'est_hourly.paruqet'* and *'pjm_hourly_est.csv'* are not used). In our next step, we will be reading these files and pre-processing these data in this order:
- Getting the time data of each individual time step and generalizing them
    - Hour of the day *i.e. 0-23*
    - Day of the week *i.e. 1-7*
    - Month *i.e. 1-12*
    - Day of the year *i.e. 1-365*
    
    
- Scale the data to values between 0 and 1
    - Algorithms tend to perform better or converge faster when features are on a relatively similar scale and/or close to normally distributed
    - Scaling preserves the shape of the original distribution and doesn't reduce the importance of outliers.
    
    
- Group the data into sequences to be used as inputs to the model and store their corresponding labels
    - The **sequence length** or **lookback period** is the number of data points in history that the model will use to make the prediction
    - The label will be the next data point in time after the last one in the input sequence
    

- The inputs and labels will then be split into training and test sets

In [80]:
#Un Normalize data

xforcemin=-10
xforcemax=10
yforcemin=-10
yforcemax=10

zforcemin=-15
zforcemax=10
rolltorquemin=-2
rolltorquemax=2
pitchtorquemin=-2
pitchtorquemax=2
yawtorquemin=-.1
yawtorquemax=.1

xforce_normalized=-0.014937

xforce_original=(((xforce_normalized+1)/2)*(xforcemax-xforcemin)+xforcemin)
print(xforce_original)
xforce_normalized2=((xforce_original-xforcemin)/(xforcemax-xforcemin)*2)-1

print(xforce_normalized2)

-0.14936999999999934
-0.014936999999999978


In [ ]:
normalizeddataframe=pd.read_csv('forcetorquebuttonresults_09_21_2021.csv')#.head()
originaldataframe=normalizeddataframe
#unnormalizerange=range(0,5,575)
#print(unnormalizerange)

for i in range(0,575,6):
  
    originaldataframe.iloc[i]=(((normalizeddataframe.iloc[i]+1)/2)*(xforcemax-xforcemin)+xforcemin)
    originaldataframe.iloc[i+1]=(((normalizeddataframe.iloc[i+1]+1)/2)*(yforcemax-yforcemin)+yforcemin)
    originaldataframe.iloc[i+2]=(((normalizeddataframe.iloc[i+2]+1)/2)*(zforcemax-zforcemin)+zforcemin)
    originaldataframe.iloc[i+3]=(((normalizeddataframe.iloc[i+3]+1)/2)*(rolltorquemax-rolltorquemin)+rolltorquemin)
    originaldataframe.iloc[i+4]=(((normalizeddataframe.iloc[i+4]+1)/2)*(pitchtorquemax-pitchtorquemin)+pitchtorquemin)
    

originaldataframe
originaldataframe.to_csv('forcetorquebuttonresults_unnormalized_09_21_2021.csv', index=False)  
print("data output to file")


In [106]:
#Find new min and max values for each set of data

originaldataframe=pd.read_csv('forcetorquebuttonresults_unnormalized_09_21_2021.csv')#.head()

xforcemin=999
xforcemax=-999
yforcemin=999
yforcemax=-999
zforcemin=999
zforcemax=-999
rolltorquemin=999
rolltorquemax=-999
pitchtorquemin=999
pitchtorquemax=-999


for i in range(0,575,6):
    xtempmin=originaldataframe.iloc[i].min(axis=0)
    xtempmax=originaldataframe.iloc[i].max(axis=0)
    ytempmin=originaldataframe.iloc[i+1].min(axis=0)
    ytempmax=originaldataframe.iloc[i+1].max(axis=0)
    ztempmin=originaldataframe.iloc[i+2].min(axis=0)
    ztempmax=originaldataframe.iloc[i+2].max(axis=0)
    rolltempmin=originaldataframe.iloc[i+3].min(axis=0)
    rolltempmax=originaldataframe.iloc[i+3].max(axis=0)
    pitchtempmin=originaldataframe.iloc[i+4].min(axis=0)
    pitchtempmax=originaldataframe.iloc[i+4].max(axis=0)
    
    if xtempmin<xforcemin:
        xforcemin=xtempmin
    if ytempmin<yforcemin:
        yforcemin=ytempmin
    if ztempmin<zforcemin:
        zforcemin=ztempmin
        #print("row",i+2,"new min =",zforcemin)
    #if ztempmin<-50:
        #print("row",i+2,"low z =",zforcemin)
        
    if rolltempmin<rolltorquemin:
        rolltorquemin=rolltempmin
   
        
    if xtempmax>xforcemax:
        xforcemax=xtempmax
    if ytempmax>yforcemax:
        yforcemax=ytempmax
    if ztempmax>zforcemax:
        zforcemax=ztempmax
        #print("new max=",zforcemax)
    if rolltempmax>rolltorquemax:
        rolltorquemax=rolltempmax

    if pitchtempmin<pitchtorquemin:
        pitchtorquemin=pitchtempmin    
        
    if pitchtempmax>pitchtorquemax:
        pitchtorquemax=pitchtempmax   
   
 
    
print("xforcemin=",xforcemin,"xforcemax=",xforcemax)
print("yforcemin=",yforcemin,"yforcemax=",yforcemax)
print("zforcemin=",zforcemin,"zforcemax=",zforcemax)
print("rolltorquemin=",rolltorquemin,"rolltorquemax=",rolltorquemax)
print("pitchtorquemin=",pitchtorquemin,"pitchtorquemax=",pitchtorquemax)






xforcemin= -22.0667781829834 xforcemax= 21.32257652282715
yforcemin= -33.593490600585945 yforcemax= 21.48625946044922
zforcemin= -134.9068450927734 zforcemax= 7.664425373077393
rolltorquemin= -4.071350574493408 rolltorquemax= 5.534818649291992
pitchtorquemin= -5.719254970550537 pitchtorquemax= 5.162374019622803


In [ ]:
#Create quantile transformers

originaldataframe=pd.read_csv('forcetorquebuttonresults_unnormalized_09_21_2021.csv')#.head()

xforcemin=999
xforcemax=-999
yforcemin=999
yforcemax=-999
zforcemin=999
zforcemax=-999
rolltorquemin=999
rolltorquemax=-999
pitchtorquemin=999
pitchtorquemax=-999


for i in range(0,575,6):
    xtempmin=originaldataframe.iloc[i].min(axis=0)
    xtempmax=originaldataframe.iloc[i].max(axis=0)
    ytempmin=originaldataframe.iloc[i+1].min(axis=0)
    ytempmax=originaldataframe.iloc[i+1].max(axis=0)
    ztempmin=originaldataframe.iloc[i+2].min(axis=0)
    ztempmax=originaldataframe.iloc[i+2].max(axis=0)
    rolltempmin=originaldataframe.iloc[i+3].min(axis=0)
    rolltempmax=originaldataframe.iloc[i+3].max(axis=0)
    pitchtempmin=originaldataframe.iloc[i+4].min(axis=0)


In [108]:
#originaldataframe

In [107]:
#print(test1)
#print(normalizeddataframe.iloc[0])

In [31]:
normalizeddataframe

,header0,header1,header2,header3,header4,header5,header6,header7,header8,header9,...,header20,header21,header22,header23,header24,header25,header26,header27,header28,header29
0,-0.001494,-0.006114,-0.002769,0.002177,-0.002500,-0.004409,-0.004502,-0.004180,-0.000122,-0.004193,...,-0.104987,-0.047108,-0.112094,-0.100577,-0.040931,-0.058437,-0.100992,-0.085715,-0.083076,-0.019339
1,0.029401,0.031186,0.054998,0.044918,0.068072,0.026208,0.025993,0.035039,0.048243,0.062763,...,-0.167204,-0.154656,-0.159713,-0.431183,-0.340838,-0.326700,-0.343063,-0.458384,-0.504819,-0.386363
2,0.160614,0.170542,0.134455,0.150562,0.137968,0.162312,0.160939,0.163658,0.156357,0.158699,...,-1.691840,-1.771973,-1.708544,-1.643269,-1.698463,0.029310,0.120428,0.075578,0.168152,0.029451
3,-0.021709,-0.026145,-0.034432,-0.034932,-0.036271,-0.033615,-0.026524,-0.028578,-0.032416,-0.030951,...,0.109933,0.135964,0.122471,0.274029,0.290201,0.271022,0.249000,0.339047,0.354345,0.325709
4,-0.001143,0.002904,-0.000305,0.005120,0.002741,-0.003509,-0.000297,-0.004837,0.003761,0.001875,...,-1.309165,-0.800369,-1.300103,-1.264863,-0.705196,-0.849624,-1.253531,-1.147455,-1.114735,-0.571323


In [4]:
# The scaler objects will be stored in this dictionary so that our output test data from the model can be re-scaled during evaluation
label_scalers = {}

train_x = []
test_x = {}
test_y = {}

for file in tqdm_notebook(os.listdir(data_dir)): 
    # Skipping the files we're not using
    if file[-4:] != ".csv" or file == "pjm_hourly_est.csv":
        continue
    
    # Store csv file in a Pandas DataFrame
    df = pd.read_csv(data_dir + file, parse_dates=[0])
    # Processing the time data into suitable input formats
    df['hour'] = df.apply(lambda x: x['Datetime'].hour,axis=1)
    df['dayofweek'] = df.apply(lambda x: x['Datetime'].dayofweek,axis=1)
    df['month'] = df.apply(lambda x: x['Datetime'].month,axis=1)
    df['dayofyear'] = df.apply(lambda x: x['Datetime'].dayofyear,axis=1)
    df = df.sort_values("Datetime").drop("Datetime",axis=1)
    
    # Scaling the input data
    sc = MinMaxScaler()
    label_sc = MinMaxScaler()
    data = sc.fit_transform(df.values)
    # Obtaining the Scale for the labels(usage data) so that output can be re-scaled to actual value during evaluation
    label_sc.fit(df.iloc[:,0].values.reshape(-1,1))
    label_scalers[file] = label_sc
    
    # Define lookback period and split inputs/labels
    lookback = 90
    inputs = np.zeros((len(data)-lookback,lookback,df.shape[1]))
    labels = np.zeros(len(data)-lookback)
    
    for i in range(lookback, len(data)):
        inputs[i-lookback] = data[i-lookback:i]
        labels[i-lookback] = data[i,0]
    inputs = inputs.reshape(-1,lookback,df.shape[1])
    labels = labels.reshape(-1,1)
    
    # Split data into train/test portions and combining all data from different files into a single array
    test_portion = int(0.1*len(inputs))
    if len(train_x) == 0:
        train_x = inputs[:-test_portion]
        train_y = labels[:-test_portion]
    else:
        train_x = np.concatenate((train_x,inputs[:-test_portion]))
        train_y = np.concatenate((train_y,labels[:-test_portion]))
    test_x[file] = (inputs[-test_portion:])
    test_y[file] = (labels[-test_portion:])

/home/scott/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [6]:
print(train_x.shape)

(980185, 90, 5)


We have a total of 980,185 sequences of training data

To improve the speed of our training, we can process the data in batches so that the model does not need to update its weights as frequently. The Torch *Dataset* and *DataLoader* classes are useful for splitting our data into batches and shuffling them.

In [7]:
batch_size = 1024

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

We can also check if we have any GPUs to speed up our training time by many folds. If you’re using FloydHub with GPU to run this code, the training time will be significantly reduced.

In [16]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
print(device)

cuda


In [17]:
device = torch.device("cpu")

Next, we'll be defining the structure of the GRU and LSTM models. Both models have the same structure, with the only difference being the **recurrent layer** (GRU/LSTM) and the initializing of the hidden state. The hidden state for the LSTM is a tuple containing both the **cell state** and the **hidden state**, whereas the GRU only has a single hidden state.

In [9]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

The training process is defined in a function below so that we can reproduce it for both models. Both models will have the same number of **dimensions** in the *hidden state* and *layers*, trained over the same number of **epochs** and **learning rate**, and trained and tested on the exact same set of data.

For the purpose of comparing the performance of both models as well, we'll being tracking the time it takes for the model to train and eventually comparing the final accuracy of both models on the test set. For our accuracy measure, we'll use *Symmetric Mean Absolute Percentage Error (sMAPE)* to evaluate the models. *sMAPE* is the sum of the **absolute difference** between the predicted and actual values divided by the average of the predicted and actual value, therefore giving a percentage measuring the amount of error. 

This is the formula for *sMAPE*:

$sMAPE = \frac{100%}{n} \sum_{t=1}^n \frac{|F_t - A_t|}{(|F_t + A_t|)/2}$

In [10]:
def train(train_loader, learn_rate, hidden_dim=256, EPOCHS=5, model_type="GRU"):
    
    # Setting common hyperparameters
    input_dim = next(iter(train_loader))[0].shape[2]
    output_dim = 1
    n_layers = 2
    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.clock()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])
            model.zero_grad()
            
            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.clock()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Time Elapsed for Epoch: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

def evaluate(model, test_x, test_y, label_scalers):
    model.eval()
    outputs = []
    targets = []
    start_time = time.clock()
    for i in test_x.keys():
        inp = torch.from_numpy(np.array(test_x[i]))
        labs = torch.from_numpy(np.array(test_y[i]))
        h = model.init_hidden(inp.shape[0])
        out, h = model(inp.to(device).float(), h)
        outputs.append(label_scalers[i].inverse_transform(out.cpu().detach().numpy()).reshape(-1))
        targets.append(label_scalers[i].inverse_transform(labs.numpy()).reshape(-1))
    print("Evaluation Time: {}".format(str(time.clock()-start_time)))
    sMAPE = 0
    for i in range(len(outputs)):
        sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
    print("sMAPE: {}%".format(sMAPE*100))
    return outputs, targets, sMAPE

In [11]:
lr = 0.001
gru_model = train(train_loader, lr, model_type="GRU")

Starting Training of GRU model
Epoch 1......Step: 200/957....... Average Loss for Epoch: 0.006068045466963668
Epoch 1......Step: 400/957....... Average Loss for Epoch: 0.00339295689118444
Epoch 1......Step: 600/957....... Average Loss for Epoch: 0.002416420340402207
Epoch 1......Step: 800/957....... Average Loss for Epoch: 0.0018970439098484348
Epoch 1/5 Done, Total Loss: 0.0016335768315337974
Time Elapsed for Epoch: 180.094723 seconds
Epoch 2......Step: 200/957....... Average Loss for Epoch: 0.0002500653800962027
Epoch 2......Step: 400/957....... Average Loss for Epoch: 0.00024024876929615858
Epoch 2......Step: 600/957....... Average Loss for Epoch: 0.00023239032923205136
Epoch 2......Step: 800/957....... Average Loss for Epoch: 0.00022369524755049496
Epoch 2/5 Done, Total Loss: 0.00021872715933283436
Time Elapsed for Epoch: 180.25017699999998 seconds
Epoch 3......Step: 200/957....... Average Loss for Epoch: 0.00017353565825033003
Epoch 3......Step: 400/957....... Average Loss for Epo

In [19]:
lstm_model = train(train_loader, lr, model_type="LSTM")

Starting Training of LSTM model


KeyboardInterrupt: 

As we can see from the training time of both models, the GRU model is the clear winner in terms of speed, as we have mentioned earlier. The GRU finished 5 training epochs 72 seconds faster than the LSTM model.

Moving on to measuring the accuracy of both models, we’ll now use our evaluate() function and test dataset.

In [12]:
gru_outputs, targets, gru_sMAPE = evaluate(gru_model, test_x, test_y, label_scalers)

Evaluation Time: 4.838130999999976
sMAPE: 0.2644234794074204%


In [18]:
lstm_outputs, targets, lstm_sMAPE = evaluate(lstm_model, test_x, test_y, label_scalers)

RuntimeError: Input and parameter tensors are not at the same device, found input tensor at cpu and parameter tensor at cuda:0

While the LSTM model may have made smaller errors and edged the GRU model slightly in terms of performance accuracy, the difference is insignificant and thus inconclusive. There have been many other tests conducted by others comparing both these models but there has largely been no clear winner as to which is the better architecture overall.